In [1]:
import os
os.chdir("../")
os.getcwd()

'/home/izam/coding/GemInsights'

In [2]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PromptingConfig:
    root_dir: Path
    model_name: str
    response_file_name: str
    candidates_file_name: str
    credentials: Path
    generation_config: dict
    project_name: str
    project_location: str
    prompt_file_path: Path
    images_file_path: Path
    

In [3]:
from gemInsights.constants import *
from gemInsights.utils.common import read_yaml, create_directories, load_json

In [4]:
#udpate configuration manager
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        credentials_file_path = CREDENTIALS_FILE_PATH,
        params_file_path = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.credentials = credentials_file_path
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])


    def get_promting_config(self) -> PromptingConfig:
        config = self.config.prompting
        
        create_directories([config.root_dir])

        prompting_config = PromptingConfig(
            root_dir=config.root_dir,
            model_name=config.model_name, 
            response_file_name=config.response_file_name,
            candidates_file_name=config.candidates_file_name,
            credentials=self.credentials,
            generation_config=dict(self.params.generation_config),
            project_name=config.project_name,
            project_location=config.project_location,
            prompt_file_path=config.prompt_file_path,
            images_file_path=config.images_file_path,
        )

        return prompting_config

In [5]:
from google.cloud import aiplatform
from gemInsights import logger
import os
import base64
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part
from gemInsights.utils.common import load_json, save_json, load_bin, read_text
from trulens_eval import Feedback, Tru, LiteLLM, Huggingface, TruBasicApp
from trulens_eval.feedback import Groundedness

In [6]:
class Prompting:
    def __init__(self, config: PromptingConfig):
        self.config = config

    def _setup_env(self):
        tru = Tru()
        # tru.reset_database()
        logger.info(f"initialized trulens with db")

        aiplatform.init(
            project = self.config.project_name,
            location= self.config.project_location,
        )
        logger.info(f"Google cloud project name - {self.config.project_name}")
        
        os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = self.config.credentials
        logger.info("loaded the google cloud credentials")

        self.model = GenerativeModel(self.config.model_name)
        logger.info(f"using the model - {self.config.model_name}")


    def _set_feedback(self):
        provider = LiteLLM(model_engine='chat-bison-32k', max_output_tokens=2048, temperature=0.0)
        grounded = Groundedness(groundedness_provider=provider)
        
        # LLM-based feedback functions
        f_criminality = Feedback(
            provider.criminality_with_cot_reasons,
            name="Criminality",
            higher_is_better=False,
        ).on_output()

        f_insensitivity = Feedback(
            provider.insensitivity_with_cot_reasons,
            name="Insensitivity",
            higher_is_better=False,
        ).on_output()

        f_maliciousness = Feedback(
            provider.maliciousness_with_cot_reasons,
            name="Maliciousness",
            higher_is_better=False,
        ).on_output()


        f_hate = Feedback(
            provider.harmfulness_with_cot_reasons,
            name="Harmfulness",
            higher_is_better=False
        ).on_output()

        f_controvertial = Feedback(
            provider.controversiality_with_cot_reasons,
            name="Coherence",
            higher_is_better=False,
        ).on_output()


        f_coherence = Feedback(
            provider.coherence_with_cot_reasons,
            name="Coherence",
            higher_is_better=True,
        ).on_output()


        f_currectness = Feedback(
            provider.correctness_with_cot_reasons,
            name="Currectness",
            higher_is_better=True,
        ).on_output()

        f_helpful = Feedback(
            provider.helpfulness_with_cot_reasons,
            name="Helpfulness",
            higher_is_better=True
        ).on_output()

        f_conciseness = Feedback(
            provider.conciseness_with_cot_reasons,
            name="Conciseness",
            higher_is_better=True,
        ).on_output()


        self.all_feedbacks = [
            f_coherence,
            f_currectness,
            f_helpful,
            f_conciseness,
            f_criminality,
            f_insensitivity,
            f_maliciousness,
            f_hate,
            f_controvertial,
        ]

    def _initiate_data(self):
        print(f"{self.config.prompt_file_path}")
        self.prompt = read_text(Path(self.config.prompt_file_path))
        self.images = load_bin(Path(self.config.images_file_path))


    def get_response(self):
        self._setup_env()
        self._initiate_data()
        self._set_feedback()
        
        model = self.model
        images = self.images
        logger.info(type(self.images))
        configuration = self.config.generation_config

        def _llm_standalone(prompt):
            return model.generate_content(
                [prompt]+images , generation_config=configuration).text
        
        tru_app_recorder = TruBasicApp(_llm_standalone, app_id="Sentiment bot", feedbacks=self.all_feedbacks)
        logger.info("created the basic recorder app")

        logger.info(f"generating response with config - {self.config.generation_config}")
        with tru_app_recorder as records:
            response = tru_app_recorder.app("generate something")
        
        save_json(path=os.path.join(self.config.root_dir, self.config.response_file_name), data={"response": response})
        logger.info(response)

In [ ]:
try:
    config = ConfigurationManager()
    prompting_config = config.get_promting_config()
    prompting_config = Prompting(config=prompting_config)
    prompting_config.get_response()
except Exception as e:
    raise e

[2023-12-22 03:11:39,960: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-12-22 03:11:39,962: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-22 03:11:39,962: INFO: common: created directory at: artifacts]
[2023-12-22 03:11:39,963: INFO: common: created directory at: artifacts/prompting]
🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.
[2023-12-22 03:11:39,971: INFO: 654671516: initialized trulens with db]
[2023-12-22 03:11:39,972: INFO: 654671516: Google cloud project name - ultra-heading-407815]
[2023-12-22 03:11:39,972: INFO: 654671516: loaded the google cloud credentials]
[2023-12-22 03:11:39,973: INFO: 654671516: using the model - gemini-pro-vision]
artifacts/prompt_generation/prompt.txt
[2023-12-22 03:11:39,974: INFO: common: text file loaded from: artifacts/prompt_generation/prompt.txt]
[2023-12-22 03:11:39,975: INFO: com

[2023-12-22 03:12:04,003: INFO: migration: Context impl SQLiteImpl.]
[2023-12-22 03:12:04,004: INFO: migration: Will assume non-transactional DDL.]
[2023-12-22 03:12:04,015: INFO: sqlalchemy_db: ✅ feedback result Currectness DONE feedback_result_hash_7b20354c1443b77ef3c14d44b5f115cd]
[2023-12-22 03:12:04,045: INFO: migration: Context impl SQLiteImpl.]
[2023-12-22 03:12:04,046: INFO: migration: Will assume non-transactional DDL.]
[2023-12-22 03:12:04,051: INFO: sqlalchemy_db: ✅ feedback result Conciseness DONE feedback_result_hash_3b33c1a154f2d832e89c257763ab6cff]
[2023-12-22 03:12:04,695: INFO: migration: Context impl SQLiteImpl.]
[2023-12-22 03:12:04,696: INFO: migration: Will assume non-transactional DDL.]
[2023-12-22 03:12:04,701: INFO: sqlalchemy_db: ✅ feedback result Helpfulness DONE feedback_result_hash_6a70acf5d10074cb238d4833656232cc]
[2023-12-22 03:12:04,999: INFO: migration: Context impl SQLiteImpl.]
[2023-12-22 03:12:04,999: INFO: migration: Will assume non-transactional DDL